
# Tutorial: Grid Search CV: The Exhaustive Search

Selamat datang di subchapter 3.2! Setelah kita memahami perbedaan antara *parameter* dan *hyperparameter*, sekarang saatnya belajar cara menemukan nilai hyperparameter terbaik secara sistematis.

Metode pertama yang akan kita pelajari adalah **Grid Search CV**. Sesuai namanya, metode ini bekerja dengan cara yang sangat teliti dan lengkap: ia akan mencoba **setiap kemungkinan kombinasi** dari hyperparameter yang kita berikan dan mengevaluasinya menggunakan *Cross-Validation* (CV).

Di notebook ini, kita akan membedah cara kerja `GridSearchCV` dari Scikit-learn dari awal hingga akhir.




---
### 1. Tujuan Pembelajaran

Di akhir notebook ini, Anda akan dapat:

* Menjelaskan konsep di balik Grid Search.
* Mendefinisikan "grid" hyperparameter menggunakan dictionary Python.
* Mengimplementasikan `GridSearchCV` untuk menemukan kombinasi hyperparameter terbaik.
* Mengakses dan menginterpretasikan hasil dari `GridSearchCV`, termasuk parameter dan skor terbaik.




---
### 2. Analogi: Koki yang Perfeksionis

Bayangkan Anda seorang koki yang ingin menemukan resep sup yang paling sempurna. Anda memiliki dua "tombol" (hyperparameter) yang bisa diatur:

1.  **Tingkat Garam:** `[1 sendok teh, 2 sendok teh, 3 sendok teh]`
2.  **Tingkat Merica:** `[0.5 sendok teh, 1 sendok teh]`

Sebagai koki yang teliti, Anda memutuskan untuk mencoba **setiap kemungkinan kombinasi**:
* Garam 1 sdt, Merica 0.5 sdt
* Garam 1 sdt, Merica 1 sdt
* Garam 2 sdt, Merica 0.5 sdt
* Garam 2 sdt, Merica 1 sdt
* Garam 3 sdt, Merica 0.5 sdt
* Garam 3 sdt, Merica 1 sdt



Inilah yang dilakukan **Grid Search**: ia membuat "grid" dari semua kemungkinan dan secara metodis menguji setiap sel di dalam grid tersebut untuk menemukan yang terbaik.




---
### 3. Setup: Data dan Model

Kita akan menggunakan dataset California Housing dan model `Ridge Regression`. Hyperparameter yang akan kita *tuning* adalah `alpha`.



In [3]:
import pandas as pd
import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

In [4]:
housing = fetch_california_housing()
X,y = housing.data, housing.target
X_train,X_test,y_train,y_test = train_test_split(X,y, random_state=42)

HTTPError: HTTP Error 403: Forbidden


---
### 4. Implementasi `GridSearchCV`

Prosesnya terdiri dari 4 langkah utama.




#### Langkah 1: Buat Pipeline Model

Sangat penting untuk memasukkan `StandardScaler` ke dalam pipeline. Ini memastikan bahwa di setiap *fold* cross-validation, data di-scale dengan benar untuk mencegah *data leakage*.



In [ ]:
pipe = make_pipeline(
	StandardScaler(),
	Ridge()
)


#### Langkah 2: Definisikan Grid Hyperparameter

Kita membuat sebuah *dictionary* di mana *key* adalah nama hyperparameter dalam pipeline dan *value* adalah daftar nilai yang ingin kita coba.

**Penting:** Nama hyperparameter harus diawali dengan nama langkah model di pipeline (dalam huruf kecil), diikuti oleh dua garis bawah (`__`). Dalam kasus `make_pipeline`, nama langkahnya sama dengan nama kelas modelnya (`ridge`). Jadi, `alpha` menjadi `ridge__alpha`.



In [ ]:
param_grid = {
	'ridge__alpha': [0.001,0.01,0.1,1,10,100,1000]
}


#### Langkah 3: Inisialisasi `GridSearchCV`

Kita membuat objek `GridSearchCV` dan memberikan pipeline, grid parameter, dan beberapa pengaturan lain.



In [ ]:
grid_search = GridSearchCV(pipe, param_grid, cv=5, scoring='r2',n_jobs=-1)


#### Langkah 4: Jalankan Pencarian dengan `.fit()`

Ini adalah langkah yang paling intensif secara komputasi. `GridSearchCV` sekarang akan melatih model berkali-kali.

**Perhitungan:** 7 nilai `alpha` × 5 *folds* = **35 kali pelatihan model!**



In [ ]:
grid_search.fit(X_train,y_train)

,estimator,"Pipeline(step...e', Ridge())])"
,param_grid,"{'ridge__alpha': [0.001, 0.01, ...]}"
,scoring,'r2'
,n_jobs,-1
,refit,True
,cv,5
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,copy,True



---
### 5. Menganalisis Hasil

Setelah proses `.fit()` selesai, `GridSearchCV` menyimpan semua informasi yang kita butuhkan.




#### Mengakses Kombinasi Terbaik

Kita bisa melihat hyperparameter terbaik dan skor CV terbaik yang dihasilkannya.



In [ ]:
print(f"kombinasi hyperparameter terbaik: {grid_search.best_params_}")
print(f"Skor R2 cross-validation terbaik: {grid_search.best_score_:.4f}" )

kombinasi hyperparameter terbaik: {'ridge__alpha': 0.001}
Skor R2 cross-validation terbaik: 0.6072



**Interpretasi:**

`GridSearchCV` menemukan bahwa `alpha=0.001` adalah nilai terbaik di antara pilihan yang kita berikan, menghasilkan skor R² rata-rata sekitar **0.6072** selama 5-fold cross-validation.




#### Mengakses Model Terbaik

`GridSearchCV` tidak hanya memberi tahu kita parameter terbaik, tetapi juga secara otomatis melatih ulang sebuah model akhir pada **keseluruhan data pelatihan** menggunakan parameter terbaik tersebut. Model ini dapat diakses melalui atribut `.best_estimator_`.



In [ ]:
best_model = grid_search.best_estimator_
print("model terbaik yang sudah dilatih:\n", best_model)

model terbaik yang sudah dilatih:
 Pipeline(steps=[('standardscaler', StandardScaler()),
                ('ridge', Ridge(alpha=0.001))])



---
### 6. Evaluasi Akhir pada Test Set

Sekarang kita bisa menggunakan `best_model` ini untuk membuat prediksi dan mengevaluasinya pada data pengujian yang belum pernah dilihat sebelumnya.



In [ ]:
test_score = best_model.score(X_test,y_test)
print(f"skor r2 pada test set: {test_score:.4f}")

skor r2 pada test set: 0.5911



---
### 7. Kesimpulan Tutorial

* **Grid Search** adalah metode yang kuat dan teliti untuk menemukan hyperparameter terbaik dari sekumpulan pilihan yang terbatas.
* Prosesnya: **Definisikan model -> Definisikan grid -> Inisialisasi GridSearchCV -> Fit**.
* **Kelebihan:** Dijamin menemukan kombinasi optimal di dalam grid yang Anda berikan.
* **Kekurangan:** Menjadi sangat lambat jika Anda memiliki banyak hyperparameter atau banyak nilai untuk diuji.

Di subchapter selanjutnya, kita akan belajar `RandomizedSearchCV`, sebuah alternatif yang lebih efisien untuk ruang pencarian yang besar.
